# CDPH variant totals by type

By [Matt Stiles](https://www.latimes.com/people/matt-stiles)

Downloads [variant totals](https://www.cdph.ca.gov/Programs/CID/DCDC/Pages/COVID-19/COVID-Variants.aspx) published by the California Department of Public Health.

## Import

Code formatting with [black](https://pypi.org/project/nb-black/).

In [1]:
%load_ext lab_black

Import dependencies.

In [ ]:
import os
import pytz
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import pandas as pd
import lxml
import glob

In [3]:
tz = pytz.timezone("America/Los_Angeles")

In [4]:
today = datetime.now(tz).date()

## Scrape

### Get url for CDPH's variants summary page

In [5]:
url = "https://www.cdph.ca.gov/Programs/CID/DCDC/Pages/COVID-19/COVID-Variants.aspx"

### Read the data on the page

In [6]:
# page = pd.read_html(url, header=0)

In [7]:
response = requests.get(url)

### Get the 'known variants of concern in California' table

In [46]:
df1 = pd.read_html(response.text, attrs={"class": "ms-rteTable-default"}, header=0)[0]

In [47]:
df1["update_date"] = today

In [48]:
df1.rename(
    columns={
        "Variant": "variant_name",
        "WHO Label": "who_label",
        "Number of Cases Caused by Variant": "cases_caused_by_variant",
        "update_date": "update_date",
    },
    inplace=True,
)

In [49]:
df1.head()

,​Variant,​WHO Label,​Number of Cases Caused by Variant,update_date
0,​B.1.1.7,​Alpha,"​10,942",2021-07-14
1,​B.1.351,​Beta,​133,2021-07-14
2,​P.1,​Gamma,"​1,774",2021-07-14
3,​B.1.617.2*,Delta​,"​1,085",2021-07-14


### Get the 'known variants of interest in California' table

In [58]:
df2 = pd.read_html(response.text, attrs={"class": "ms-rteTable-default"}, header=0)[1]

In [51]:
df2["update_date"] = today

In [64]:
df2.rename(
    columns={
        "Variant": "variant_name",
        "WHO Label": "who_label",
        "Number of Cases Caused by Variant": "cases_caused_by_variant",
        "update_date": "update_date",
    },
    inplace=True,
)

In [65]:
df2.head()

,​Variant,​WHO Label,​Number of Cases Caused by Variant
0,​B.1.427 and B.1.429,Epsilon,23464
1,​P.2,Zeta,​91
2,​B.1.525,​Eta,​56
3,B.1.526,​Iota,"​1,579"
4,​B.1.617.1,Kappa,​61


### Get the 'proportion of variants of concern and variants of interest in California change over time'

In [66]:
df3 = pd.read_html(response.text, attrs={"class": "ms-rteTable-default"}, header=0)[2]

In [67]:
df3["update_date"] = today

In [68]:
df3.rename(
    columns={
        "Specimen Collection Month": "specimen_collection_month",
        "Alpha": "alpha",
        "Beta": "beta",
        "Gamma": "gamma",
        "Delta": "delta",
        "Epsilon": "epsilon",
        "Zeta": "zeta",
        "Eta": "eta",
        "Iota": "iota",
        "Kappa": "kappa",
        "B.1.617.3": "b.1.617.3",
        "update_date": "update_date",
    },
    inplace=True,
)

In [69]:
df3.head()

,specimen_collection_month,alpha,beta,gamma,delta,epsilon,zeta,eta,iota,kappa,B.1.617.3​,update_date
0,​21-Jun,30.6%,0.2%,15.4%,42.9%,1.2%,0.0%,0.0%,6.4%,0.1%,0.0%,2021-07-14
1,​21-May,57.6%,0.3%,11.5%,5.8%,5.0%,0.0%,0.2%,10.7%,0.1%,0.0%,2021-07-14
2,21-Apr,50.3%,0.7%,8.7%,2.2%,17.1%,0.0%,0.3%,7.5%,0.4%,0.0%,2021-07-14
3,21-Mar,22.0%,0.4%,2.3%,0.0%,50.6%,0.3%,0.1%,2.6%,0.1%,0.0%,2021-07-14
4,21-Feb,5.7%,0.0%,0.0%,0.0%,59.3%,0.1%,0.0%,0.3%,0.0%,0.0%,2021-07-14


## Export

Save out the data as CSVs that are datestamped to California time.

In [70]:
data_dir = os.path.join(os.path.abspath(""), "data")

In [20]:
df1.to_csv(
    os.path.join(data_dir, f"raw/concern/variants_of_concern_ca_{today}_.csv"),
    index=False,
)

In [21]:
df2.to_csv(
    os.path.join(data_dir, f"raw/interest/variants_of_interest_ca_{today}_.csv"),
    index=False,
)

In [71]:
df3.to_csv(
    os.path.join(data_dir, f"raw/proportion/proportion_over_time_ca_{today}_.csv"),
    index=False,
)

### Concatenate

In [ ]:
path = ""
of_concern_files = glob.glob(os.path.join(path, "data/raw/concern/*.csv"))
of_interest_files = glob.glob(os.path.join(path, "data/raw/interest/*.csv"))
proportion_files = glob.glob(os.path.join(path, "data/raw/proportion/*.csv"))

Create concern/interest and proportion column dicts.

In [32]:
concern_interest_cols = {
    "​Variant": "variant_name",
    "​WHO Label": "who_label",
    "​Number of Cases Caused by Variant": "cases_caused_by_variant",
}

In [37]:
proportion_cols = {
    "Specimen Collection Month": "specimen_collection_month",
    "Alpha": "alpha",
    "Beta": "beta",
    "Gamma": "gamma",
    "Delta": "delta",
    "Epsilon": "epsilon",
    "Zeta": "zeta",
    "Eta": "eta",
    "Iota": "iota",
    "Kappa": "kappa",
    "B.1.617.3": "b.1.617.3",
    "update_date": "update_date",
}

In [33]:
def concat_files(files, cols):

    file_df = (
        pd.read_csv(f, low_memory=False)
        .assign(filename=os.path.basename(f))
        .rename(columns=cols)
        for f in files
    )

    concat_df = pd.concat(
        file_df,
        ignore_index=True,
    )
    concat_df["update_date"] = pd.to_datetime(
        concat_df["update_date"].str.replace(".csv", "", regex=False)
    )
    return concat_df

In [34]:
concern_df = concat_files(of_concern_files, concern_interest_cols)

In [31]:
concern_df.to_csv("data/variants_of_concern_timeseries.csv", index=False)

In [149]:
interest_df = concat_files(of_interest_files, concern_interest_cols)

In [150]:
interest_df.to_csv("data/variants_of_interest_timeseries.csv", index=False)

In [56]:
proportion_df = concat_files(proportion_files, proportion_cols)

In [57]:
proportion_df.to_csv("data/proportion_over_time_timeseries.csv", index=False)